In [21]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import scipy.sparse

import json
import os
from collections import Counter

from time import time

%matplotlib inline

In [22]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(copy=False)

### Reading data

In [334]:
from collections import defaultdict

doc_categories = defaultdict(set)
category_docs = defaultdict(set)

category_blacklist = {u'Articles containing proofs', 
                      u'Articles created via the Article Wizard', 
                      u'Articles with example pseudocode'}

for line in file('C:/tmp/mlp/category_info.txt'):
    title, cat = line.strip().split('\t')
    title = title.decode('utf-8')
    cat = cat.decode('utf-8')
    
    if cat in category_blacklist:
        continue
    
    doc_categories[title].add(cat) 
    category_docs[cat].add(title)

print len(doc_categories), len(category_docs)

small_cats = set()

for cat, docs in category_docs.items():
    if len(docs) == 1:
        small_cats.add(cat)

print len(small_cats)

for cat in small_cats:
    for doc in category_docs[cat]:
        doc_categories[doc].remove(cat)
    del category_docs[cat]

del small_cats

for doc in doc_categories.keys():
    if len(doc_categories[doc]) == 0:
        doc_categories[doc].add(u'OTHER')
    category_docs[u'OTHER'].add(doc)

(29762, 10735)

In [337]:
def id_counter(id_list):
    cnt = Counter()
    for el in id_list:
        cnt[el[u'element']] = el[u'count']
    return cnt

def_black_list = { 'unit', 'units', 'value', 'values', 'axis', 'axes', 'factor', 'factors', 'line', 'lines',
                 'point', 'points', 'number', 'numbers', 'variable', 'variables', 'respect', 'case', 'cases',
                 'vector', 'vectors', 'element', 'elements', 'example', 
                 'integer', 'integers', 'term', 'terms', 'parameter', 'parameters', 'coefficient', 'coefficients',
                 'formula', 'times', 'product', 'matrices', 'expression', 'complex', 'real', 'zeros', 'bits',
                 'sign',
                 'if and only if',
                 'alpha', 'beta', 'gamma', 'delta', 'epsilon', 'zeta', 'eta', 'theta', 'iota', 'kappa', 'lambda', 
                 'mu', 'nu', 'xi', 'omicron', 'pi', 'rho', 'sigma', 'tau', 'upsilon', 'phi', 'chi', 'psi', 'omega'}

def valid_def(definition):
    if len(definition) <= 3:
        return False

    return definition.lower() not in def_black_list

def rel_to_dict(rels):
    res = defaultdict(list)
    for r in rels:
        if not valid_def(r['definition']):
            continue
        res[r['identifier']].append((r['definition'], r['score']))
    return res

In [338]:
root = 'C:/tmp/mlp/mlp-output/'

docs = []
titles = []
ids = []
rels = []

empty = 0
small = 0

for f in os.listdir(root): 
    for line in file(root + f):
        doc = json.loads(line)

        title = doc['title']        
        if title not in doc_categories:
            continue

        if '(disambiguation)' in title:
            continue

        id_bag = id_counter(doc['identifiers'])
        if len(id_bag) <= 1:
            if len(id_bag) == 0:
                empty = empty + 1
            else:
                small = small + 1
            continue
        
        docs.append(doc)
        titles.append(title)
        ids.append(id_bag)

        id_rels = rel_to_dict(doc['relations'])
        rels.append(id_rels)

In [339]:
empty, small

(5374, 1566)

In [340]:
N_doc = len(titles)
print N_doc

22822


In [344]:
title_idx = {title: idx for (idx, title) in enumerate(titles)}

for doc, cats in doc_categories.items():
    if doc in title_idx:
        continue
        
    for cat in cats: 
        category_docs[cat].remove(doc)
    
    del doc_categories[doc]

print len(doc_categories)

title_cats = [doc_categories[title] for title in titles]
doc_cat_flat = {}

for line in file('C:/tmp/mlp/category_info.txt'):
    title, cat = line.strip().split('\t')
    title = title.decode('utf-8')
    cat = cat.decode('utf-8')
    doc_cat_flat[title] = cat

title_cats_flat = [doc_cat_flat[title] for title in titles]
del doc_cat_flat
print len(title_cats_flat)

all_categories = list(set(title_cats_flat))
categories_idx = {cat: idx for idx, cat in enumerate(all_categories)}
title_cats_code = np.array([categories_idx[cat] for cat in title_cats_flat])

22822

### Term Selection

Let's discard infrequent identifiers

In [346]:
all_ids = Counter()

for id_cnt in ids:
    all_ids.update(id_cnt)

len(all_ids)

12898

In [347]:
infrequent = set()

min_count = 20

for (el, cnt) in all_ids.items():
    if cnt <= min_count:
        infrequent.add(el)

len(infrequent)

10660

In [348]:
del all_ids

In [349]:
for id_cnt in ids:
    for id in list(id_cnt):
        if id in infrequent:
            del id_cnt[id]

In [350]:
np.mean([len(doc_ids) for doc_ids in ids])

12.746823240732626

Let's remove most common identifiers - based on df

In [351]:
df = Counter()
for cnt in ids:
    for id in cnt:
        df[id] = df[id] + 1

In [352]:
top = 50
mc = [id for (id, cnt) in df.most_common(top) if cnt > 3000]
print ' '.join(mc)

mc = set(mc)

n t x m p d g k f R l y c r T π C P b S s N B E F X j


In [353]:
for id_cnt in ids:
    for id in list(id_cnt):
        if id in mc:
            del id_cnt[id]

In [354]:
np.mean([len(doc_ids) for doc_ids in ids])

6.9679256857418279

In [355]:
ids[0]

Counter({u't_i': 2, u't_k': 1, u'X_k': 1, u'X_i': 1, u'X_1': 1, u't_1': 1})

Let's add some debug functions for printing

In [356]:
def get_ids(title):
    if isinstance(title, (int, long)):
        return ' '.join(ids[title])
    else:
        return ' '.join(ids[title_idx[title]])

def print_ids(title):
    print get_ids(title)

In [357]:
print_ids(u'Laplace–Stieltjes transform')

Y_2 Y_1 τ_i t_i t_n ε λ F_Y L Y_n U Y Z h t_0 t_1


In [358]:
print_ids(0)

t_k t_i X_k X_i X_1 t_1


### Evaluation

In [359]:
import cluster_evaluation

In [364]:
reload(cluster_evaluation)
evaluate = cluster_evaluation.Evaluator(doc_titles=titles, doc_ids=ids, doc_ids_definitions=rels, doc_categories=title_cats)

###  Jaccard

In [238]:
ids_sets = [set(id_list) for id_list in ids]

In [239]:
def calc_jaccard(set1, set2):
    union = len(set1 | set2)
    if not union: 
        return 0.0

    inter = len(set1 & set2)
    return inter * 1.0 / union

First, to make it faster to compute the sim matrix, need to build the inverted index

In [240]:
inv_idx = {}

for (idx, id_list) in enumerate(ids):
    for id in id_list: 
        if id in inv_idx:
            inv_idx[id].append(idx)
        else:
            inv_idx[id] = [idx]

In [241]:
def docs_to_compare(doc_id):
    res = set([])
    id_list = ids[doc_id]
    for id in id_list:
        res.update(inv_idx[id])
    if doc_id in res:
        res.remove(doc_id)
    return res

In [242]:
np.mean([len(docs_to_compare(doc_id)) for doc_id in xrange(200)])

4759.9799999999996

usual algorithms like agglomerative are super long (they are super-linear, so it takes forever to compute them over the entire 

In [243]:
k_graph = 15
k_matrix = 50
sim_threshold = 0.5
shared_nn = []

jaccard_sim_matrix = scipy.sparse.dok_matrix((N_doc, N_doc))

In [244]:
t0 = time()

for i in xrange(N_doc):
    if i % 1000 == 0:
        print "iteration %d" % i

    doc_ids = np.array(list(docs_to_compare(i)))
    sim = np.zeros(len(doc_ids))

    for (idx, j) in enumerate(doc_ids):
        sim[idx] = calc_jaccard(ids_sets[i], ids_sets[j])
    
    sim_idx = sim.argsort()[-1:-k_matrix-1:-1]
    doc_ids_to_add = doc_ids[sim_idx]

    shared_nn.append(set(doc_ids_to_add[0:k_graph]))
    jaccard_sim_matrix[i, doc_ids_to_add] = sim[sim_idx]

print "done in %0.3fs." % (time() - t0)

iteration 0
iteration 1000
iteration 2000
iteration 3000
iteration 4000
iteration 5000
iteration 6000
iteration 7000
iteration 8000
iteration 9000
iteration 10000
iteration 11000
iteration 12000
iteration 13000
iteration 14000
iteration 15000
iteration 16000
iteration 17000
iteration 18000
iteration 19000
iteration 20000
iteration 21000
iteration 22000
done in 634.224s.


But it's still too long to compute, it's $O(n^2)$

In [246]:
jaccard_sim_matrix = scipy.sparse.coo_matrix(jaccard_sim_matrix)

In [247]:
row = jaccard_sim_matrix.getrow(11)
print row.data[row.data.argsort()[::-1]]
print row.indices[row.data.argsort()[::-1]]

[ 1.          1.          0.66666667  0.66666667  0.66666667  0.66666667
  0.66666667  0.66666667  0.66666667  0.66666667  0.66666667  0.66666667
  0.66666667  0.6         0.6         0.6         0.5         0.5         0.5
  0.5         0.5         0.5         0.5         0.5         0.5         0.5
  0.5         0.5         0.5         0.5         0.5         0.5         0.5
  0.5         0.5         0.5         0.5         0.5         0.5         0.5
  0.5         0.5         0.5         0.5         0.5         0.5         0.5
  0.5         0.5         0.5       ]
[20133  1370   303  2322 12775 16604 17787  9918 17956 20205 20525  2311
   664 16082 18918 17174 19629 16292 16382 16427 17492 16608 19503 17843
 15952 18414 19121 15980 13452 14846  9454   556   644  1189  2878  5879
  6142  8025  8288 10064 13693 10176 10713 11333 11738 12025 12302 12546
   549 21364]


### DB Scan 

In [248]:
import snn_dbscan

In [249]:
res = np.array(snn_dbscan.dbscan(shared_nn, 7, 6))
print Counter(res).most_common()[:50]

iteration 0
iteration 5000
iteration 10000
iteration 15000
iteration 20000
[('noise', 14375), ('16', 46), ('4', 37), ('18', 35), ('58', 33), ('2520', 33), ('2471', 32), ('2479', 30), ('2095', 29), ('2497', 29), ('244', 28), ('159', 28), ('2387', 28), ('2440', 27), ('2408', 27), ('2361', 27), ('2447', 26), ('2151', 26), ('2106', 26), ('2038', 26), ('2452', 25), ('2412', 25), ('2481', 25), ('2445', 24), ('533', 24), ('2430', 24), ('2404', 24), ('1821', 24), ('2416', 24), ('2136', 24), ('2207', 23), ('50', 23), ('2365', 23), ('338', 23), ('2128', 23), ('2483', 23), ('2460', 23), ('2325', 23), ('2453', 22), ('2462', 22), ('2382', 22), ('2389', 22), ('499', 22), ('2023', 22), ('124', 22), ('2458', 21), ('2508', 21), ('2494', 21), ('2466', 21), ('2469', 21)]


In [250]:
res[res == 'noise'] = 0
res = res.astype(int)

In [361]:
evaluate.overall_purity(res)

0.10520550346157222

In [362]:
evaluate.high_purity_clusters(res, threshold=0.75)

[{'category': u'Bayer objects',
  'cluster': 126,
  'purity': 0.9056603773584906,
  'size': 53},
 {'category': u'Stochastic processes',
  'cluster': 157,
  'purity': 0.8,
  'size': 10}]

In [392]:
reload(cluster_evaluation)
evaluate = cluster_evaluation.Evaluator(doc_titles=titles, doc_ids=ids, doc_ids_definitions=rels, doc_categories=title_cats)

In [393]:
evaluate.print_cluster(res, 157)

size: 10

Girsanov theorem (cats: Stochastic processes, Probability theorems) F_t E_P Ω X_t X_s W_t λ μ W_s σ Z_t Y_t Y_s Q W Y X_0
Stochastic differential equation (cats: Differential equations, Stochastic processes, Stochastic calculus, Stochastic differential equations) f_i g_i Z X_t X_u μ σ D x_i u X_0 B_t
Itō calculus (cats: Definitions of mathematical integration, Stochastic calculus) ξ_j ξ_k x_l x_m M Y_0 F_t X_T h_k M_t t_i H_s Ω σ_s X_t δ H_n μ X_s σ H J Y_t Y_s K x_k α_s x_j M_0 t_1 f_i π_n μ_s X_0 t_2 B_s B_t
Quadratic variation (cats: Stochastic processes) Y_0 Δ V_t t_k C_p Ω σ_s X_v X_t X_u X_s μ_s Y_s σ H Y_t M c_p u X_0 v B_s
Bessel process (cats: Stochastic processes) Z_t X_t W_t
Novikov's condition (cats: Stochastic processes, Martingale theory) Ω F_t X_t X_s W_t W_s
Semimartingale (cats: Stochastic processes, Martingale theory) Δ σ_s H X_T X_t X_0 b_s W_s M_t
Tanaka equation (cats: Equations, Stochastic differential equations) X_t sgn B_t X_s
Filtering problem (stocha

In [254]:
evaluate.print_cluster(res, 2414)

size: 6

Covariant formulation of classical electromagnetism (cats: Special relativity, Concepts in physics, Electromagnetism) D_z J_free D_x L_int E_z E_y E_x J_β M_z M_y M_x u_α E_j E_i u D q H_z μ_0 H_x H_y α γ β D_y f_α η H ρ_free P_z λ ν μ δ ρ σ τ χ_m S_y S_x J S_z M L L_field ϕ P_x x_ν p_α B_i B_j χ_e ϵ ρ_bound B_x B_y B_z ϵ_0 J_bound P_y z u_ν
Electromagnetic tensor (cats: Tensors in general relativity, Theory of relativity, Tensors, Minkowski spacetime, Electromagnetism) ϵ_0123 E_z E_y E_x ℏ E_i B_x B_y D_α γ α μ_0 β δ ϵ ν μ ρ τ G ψ J L B_i B_k q B_z ϵ_0 u
Mathematical descriptions of the electromagnetic field (cats: Mathematical physics, Electromagnetism) E_z E_y E_x σ γ_μ Γ ε_0 j_x j_y Λ j_z γ_1 γ_0 γ_3 γ_2 ∇_α γ ∇_γ ∇_β σ_1 σ_3 β δ η θ λ ν μ ρ σ_2 G φ ψ J σ_k μ_0 B_x B_y B_z ϵ_0 q α γ_k z
Classical electromagnetism and special relativity (cats: Special relativity, Electromagnetism) ϵ E_⊥ E_z E_y E_x B_∥ J_z Λ q J_y φ μ_0 α γ β δ J_x u_z λ u_x ν μ ρ D G H J B_⊥ x_β E_∥ x_α u_

In [255]:
evaluate.print_cluster(res, 2045)

size: 6

Orthotropic material (cats: Materials, Continuum mechanics) Δ_k c_1212 c_1211 ν_13 ε_12 K_11 K_13 K_12 E_i S_44 c_2333 c_3311 c_3312 d_2 d_3 d_1 C_66 σ_1 σ_3 σ_2 ε σ_4 σ_6 G_13 G_12 E_3 E_2 E_1 D H S_23 c_1111 c_1112 Δ_2 Δ_3 Δ_1 Δ_6 Δ_4 Δ_5 c_3322 c_3323 S_55 ε_11 c_3122 c_3123 c_2233 d_j c_1231 c_1233 c_2323 c_2322 c_1122 c_1123 f_i C_46 C_44 C_45 σ_12 ε_22 ε_23 σ_22 σ_23 c_3131 c_3133 c_3333 ν_12 c_3331 ε_3 ε_2 ε_1 ε_6 ε_5 ε_4 κ K c_1223 c_1222 c_2212 C_55 C_56 σ_33 ν_23 ν_21 ε_31 c_2211 ε_33 c_2222 S_33 c_2223 G_ij c_3112 c_3111 ν_31 ν_32 C_24 C_25 C_26 C_22 C_23 c_1133 f_1 c_1131 f_3 σ_5 σ f_2 J C_36 C_35 C_34 C_33 σ_31 σ_11 K_21 K_22 K_23 G_23 v S_11 S_13 S_12 C_11 η_μ ν_ij K_33 K_32 K_31 G_31 μ S_66 S_22 c_2331 c_2231 S_k c_2312 c_2311 C_15 C_14 C_16 C_13 C_12 q
Vibration of plates (cats: Continuum mechanics) x_α Δ C_n J_0 ε_22 ω_n σ_22 J_1 u_i φ C_1 C_2 M_22 λ ν μ ρ D ω ψ β x_2 x_3 W x_1 U ε_12 J_3 ε_11 B_n λ_n h σ_11 σ_12 q u_α w x_β M_12 M_11
Mindlin–Reissner plate th

In [256]:
from sklearn import metrics

In [259]:
metrics.adjusted_mutual_info_score(title_cats_code, res)

0.027101848041983765

## Minhash 

Let's use minhash to speed up computations

* https://github.com/ekzhu/datasketch

In [20]:
from datasketch.minhash import MinHash, jaccard
from hashlib import sha1

In [159]:
lsh_hashes = 8
hash_len = 4
digest_len = lsh_hashes * hash_len

In [160]:
def digest(id_list):
    m = MinHash(num_perm=digest_len)    
    for i in id_list:
        m.digest(sha1(i.encode('utf8')))
    return m

In [161]:
minhashes = [digest(id_list) for id_list in ids]

First, let's build the index

In [59]:
def get_chunks(lst, n):
    for i in xrange(0, len(lst), n):
        yield tuple(lst[i:i+n])

In [26]:
def calc_minhash_buckets(doc_id, n):
    chunks = get_chunks(minhashes[doc_id].hashvalues, n)
    return [hash(chunk) for chunk in chunks]

In [139]:
minhash_index = {}

for doc_id in xrange(len(ids)):
    for bucket in calc_minhash_buckets(doc_id, n=hash_len):
        if bucket in minhash_index:
            minhash_index[bucket].append(doc_id)
        else: 
            minhash_index[bucket] = [doc_id]

In [140]:
np.mean([len(val) for val in minhash_index.values()])

1.8484254772068531

Now can query it:

In [130]:
def query_minhash_index(query_doc_id):
    candidates = set([])
    for bucket in calc_minhash_buckets(query_doc_id, n=hash_len):
        if bucket in minhash_index:
            candidates.update(minhash_index[bucket])
    if query_doc_id in candidates:
        candidates.remove(query_doc_id)
    return candidates

In [141]:
np.mean([len(query_minhash_index(idx)) for idx in xrange(200)])

263.94

Now we can try to do jaccard:

In [142]:
N_doc = len(ids)
jaccard_sim_matrix = scipy.sparse.dok_matrix((N_doc, N_doc), dtype=np.float32)

In [149]:
t0 = time()

for i in xrange(N_doc):
    if i % 10000 == 0:
        print "iteration %d" % i
    docs = query_minhash_index(idx)
    
    for j in docs:
        if jaccard_sim_matrix[i, j] == 0.0:
            jaccard_sim_matrix[i, j] = jaccard_sim_matrix[j, i] = calc_jaccard(ids_sets[i], ids_sets[j])

print "done in %0.3fs." % (time() - t0)

iteration 0


TypeError: 'coo_matrix' object has no attribute '__getitem__'

In [150]:
jaccard_sim_matrix = scipy.sparse.coo_matrix(jaccard_sim_matrix)

True jaccard:

In [151]:
row_id = 123

In [152]:
to_compare = list(docs_to_compare(row_id))
jaccard_real = np.zeros(len(to_compare))

for j, idx_other in enumerate(to_compare):
    jaccard_real[j] = calc_jaccard(ids_sets[row_id], ids_sets[j])

In [153]:
print jaccard_real[jaccard_real.argsort()][-2:-11:-1]

print titles[row_id]
print_ids(row_id)
print 

for idx in jaccard_real.argsort()[-2:-11:-1]:
    print '%0.2f' % jaccard_real[idx], titles[to_compare[idx]], 
    print_ids(to_compare[idx])

[ 0.41176471  0.27777778  0.26666667  0.25        0.25        0.25
  0.23809524  0.23076923  0.23076923]
Newton–Euler equations
c τ F ω m α p v_cm c_y c_x c_z

0.41 AC power P_inst RMS Q_c Q_b Q_a k_1 k_2 P_b P_c P_a S_a P_avg V_RMS P S R ϕ V X Q f k j m S_c n p S_b t t_2 t_1 ω_2 ω_1
0.28 Angle of view S_1 π FOV S_2 d P f h α_d F m L α_h α α_v f_c v D
0.27 Calculus of moving surfaces Γ ∇_η ∇_α ∇_β α γ β ε δ η N_i C B S_t F N P S T Z h k j m p ∇_i t v
0.25 Conformal symmetry K_ν D m P_ρ p P_μ P_ν K_ρ x x_μ x_ν ν μ K_μ
0.25 Negation p b ¬
0.25 Permutation polynomial p_i k_t p_t q Σ k_1 k_2 α t M L α_s Z p_1 c b d g j m l o n k_i p s r k_l y x
0.24 Stokes operator w_k P_σ Ω α γ λ_1 λ_3 λ_2 C D H L R V λ λ_k g k l n u_k u t
0.23 Lagrangian mechanics j y_pend q_m V q_j q_i L_rel q_1 n x Q_j r_i r_j pend N_i N_j r_n θ λ V_i δ D F J M L F_i y S R x_2 T W x_1 L_cm λ_i m_i d g r_1 r_2 r_3 m l F_cf μ q p r t t_2 U t_1 k q_2
0.23 Differential coding z F h k m l x_i p u_0 u t y_i H y x y_0


LSH index:

In [158]:
row = jaccard_sim_matrix.getrow(row_id)

print titles[row_id]
print_ids(row_id)

order = row.data.argsort()[:-12:-1]

print 
for i, jac in zip(row.indices[order], row.data[order]):
    print '%0.2f'% jac, titles[i], 
    print_ids(i)


 Newton–Euler equations
c τ F ω m α p v_cm c_y c_x c_z

0.16 Fiducial inference all F ω l n α P observations x X t
0.16 Cavity ring-down spectroscopy c τ τ_0 C l α R ϵ t X n
0.16 Inverse Mills ratio σ g Φ m l α p ϕ t X μ
0.15 Bigraph c E k m l n p r t V Y X
0.14 Transposable integer d F N_c k j m l N p s t X n D
0.13 Truncated normal distribution Φ α β δ λ μ ξ π σ F N ϕ X Z U b g f m l p t x
0.12 Mode (statistics) σ g P θ k m l x_i α p μ_0 t X x n ν μ
0.11 Sparse matrix m l x_i p N n t X Z
0.11 Anderson–Darling test σ p d Φ F_n F m l Y_1 X_i W Y_n S t Y_i X x w n μ
0.11 Midpoint circle algorithm C E β g p h m l x_i α y_n r t y_i X y x R n Y
0.11 Cantelli's inequality Pr σ g m l p t X λ μ


## LSA

We'll remove low-freq words, but will keep high-freq (idf will take care of that)

In [260]:
ids = [id_counter(d['identifiers']) for d in docs]

all_ids = Counter()

for id_cnt in ids:
    all_ids.update(id_cnt)

infrequent = set()
min_count = 15

for (el, cnt) in all_ids.items():
    if cnt <= min_count:
        infrequent.add(el)

for id_cnt in ids:
    for id in (set(id_cnt) & infrequent):
        del id_cnt[id]

del all_ids
del infrequent

In [261]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD, randomized_svd

In [262]:
def unwrap_counter(cnt):
    res = []
    for id, c in cnt.items():
        res.extend([id] * c)
    return res

vectorizer = TfidfVectorizer(analyzer=unwrap_counter)
X = vectorizer.fit_transform(ids)

In [263]:
X.shape

(22822, 2729)

In [264]:
n_components = 100

In [265]:
U, S, Vt = randomized_svd(X, n_components)

In [266]:
U.shape, S.shape, Vt.shape

((22822L, 100L), (100L,), (100L, 2729L))

In [267]:
V = Vt.T

In [268]:
V_neg = V.sum(axis=0) < 0
V[:, V_neg] = -V[:, V_neg]

In [269]:
np.allclose(V.T.dot(V), np.eye(n_components))

True

In [270]:
terms = vectorizer.get_feature_names()
# U here is term-term matrix because we did svd on tfidf.T - i.e. on term-doc matrix, not doc-term

for topic_id in xrange(10):
    print 'topic #%d' % (topic_id + 1)
    topic = V[:, topic_id]
    indices = np.abs(topic).argsort()[::-1][:10]
    contribution = topic[indices]
    for idx, contrib in zip(np.nditer(indices), np.nditer(contribution)):
        print '%s: %0.2f,' % (terms[idx], contrib),
    print
    print

topic #1
b: 0.48, k: 0.32, f: 0.29, d: 0.29, y: 0.24, t: 0.21, m: 0.19, p: 0.17, n: 0.16, x: 0.15,

topic #2
Ω: -0.56, U: 0.47, d: 0.46, k: -0.30, f: 0.15, n: -0.12, y: 0.09, p: -0.07, m: 0.06, x: 0.05,

topic #3
S: 0.47, R: -0.40, r: -0.35, t: -0.31, k: -0.20, y: 0.17, x: 0.15, m: 0.12, p: 0.11, n: 0.11,

topic #4
T: -0.68, S: 0.23, c: 0.20, L: -0.18, G: 0.18, d: 0.15, x: 0.13, r: 0.12, t: 0.12, n: 0.11,

topic #5
F: -0.48, C: 0.31, s: 0.30, d: -0.23, S: 0.22, B: 0.17, r: -0.17, P: -0.16, X: 0.15, t: 0.15,

topic #6
ω: 0.65, θ: 0.41, z: -0.30, x: -0.15, k: 0.15, r: -0.13, X: -0.10, t: 0.10, c: 0.09, b: -0.09,

topic #7
y: -0.43, x: -0.37, R: 0.37, P: -0.30, Y: -0.23, t: -0.21, b: 0.18, r: 0.16, X: 0.13, f: 0.10,

topic #8
B: -0.50, π: 0.34, b: -0.33, k: -0.19, f: 0.19, S: -0.18, θ: -0.18, X: 0.14, t: -0.14, r: 0.14,

topic #9
α: 0.43, d: -0.42, h: 0.37, t: 0.30, r: -0.17, Y: -0.16, k: -0.16, z: 0.14, X: -0.12, f: 0.11,

topic #10
Z: -0.50, B: 0.46, T: 0.29, R: 0.19, X: -0.18, r: 0.16,

In [271]:
X_red = X.dot(V)
X_red.shape

(22822L, 100L)

Want to use cosine through euclidean, but for that need to normalize row-wise

In [272]:
np.linalg.norm(X_red[0])

0.66399702893712997

In [273]:
from sklearn.preprocessing import Normalizer
normalizer = Normalizer(copy=False)

In [274]:
X_red = normalizer.fit_transform(X_red)

In [275]:
np.linalg.norm(X_red[0])

0.99999999999999989

In [276]:
from sklearn.cluster import KMeans, MiniBatchKMeans

In [277]:
km = KMeans(n_clusters=50)

In [278]:
km.fit(X_red)

KMeans(copy_x=True, init='k-means++', max_iter=300, n_clusters=50, n_init=10,
    n_jobs=1, precompute_distances='auto', random_state=None, tol=0.0001,
    verbose=0)

In [279]:
centroids = km.cluster_centers_.dot(Vt)

In [280]:
order_centroids = centroids.argsort()[:, ::-1]
terms = vectorizer.get_feature_names()

for i in range(50):
    print "Cluster %d:" % i,
    for ind in order_centroids[i, :10]:
        print ' %s' % terms[ind],
    print

Cluster 0:  B  C  S  E  p  P  c  n  R  Ω
Cluster 1:  y  x  z  d  b  t  p  f  n  r
Cluster 2:  c  o  t  r  n  l  p  m  s  b
Cluster 3:  h  g  f  λ  n  c  x_0  k  m  t
Cluster 4:  D  C  E  T  R  P  G  L  F  o
Cluster 5:  f  x  g  t  b  n  y  d  h  Y
Cluster 6:  X  Y  f  Z  B  x  p  δ  P  t
Cluster 7:  r  d  π  t  c  n  m  x  θ  k
Cluster 8:  O  H_2  H  O_2  x_i  N  V  P  C  G
Cluster 9:  G  g  K  F  ϕ  V  t  k  r  S
Cluster 10:  T  k  S  k_B  d  R  n  μ  N  x
Cluster 11:  j  l  k  n  m  t  g  x  ℏ  σ
Cluster 12:  σ  τ  μ  t  T  n  x  j  π  θ
Cluster 13:  L  γ  μ  K  ν  n  t  g  δ  β
Cluster 14:  α  β  x  n  m  γ  k  l  ϵ  g
Cluster 15:  ρ  L  V  v  d  x  k  g  Δ  θ
Cluster 16:  x  t  p  d  n  y  m  f  u  b
Cluster 17:  F  G  x  E  S  C  X  N  k  P
Cluster 18:  b  c  x  d  n  y  f  m  p  J
Cluster 19:  x  y  n  f  d  t  b  π  m  u
Cluster 20:  k  n  t  m  p  x  π  Z  g  f
Cluster 21:  u  v  x  p  t  X  y  h  g  j
Cluster 22:  s  t  n  m  r  p  x  k  π  d
Cluster 23:  p  m  n  x  U  k  c  

In [281]:
np.bincount(km.labels_)

array([ 421,  899,  399,  309,  288,  609,  349,  644,  160,  379,  508,
        266,  266, 1268,  492,  349,  610,  313,  539,  751,  508,  423,
        450, 1177,  169,  292,  408,  364,  230,  480,  373,  337,  441,
        375,  774,  434,  548,  291,  290,  477,  418,  363,  315,  458,
        960,  455,  388,  199,  237,  369], dtype=int64)

In [282]:
evaluate.overall_purity(km.labels_)

0.054346625108437144

In [283]:
evaluate.high_purity_clusters(km.labels_, 0.5)

[]

In [284]:
metrics.adjusted_mutual_info_score(title_cats_code, km.labels_)

0.05487087256729932

##  Scatter/Gather

S/G is k-means with

- seed selection (buckshot: do hierarchical clustering on sample of $\sqrt{n}$)
- k-means, but centrod is concatenation of all docs in the cluster
- refinement operations

For tf-idf can just sum two vectors: $\text{tf}(t_i, d_1) \cdot \text{idf}(t_i) + \text{tf}(t_i, d_2) \cdot \text{idf}(t_2) = \Big( \text{tf}(t_i, d_1) + \text{tf}(t_i, d_2) \Big) \cdot \text{idf}(t_i)$

In [49]:
def buckshot():
    pass

K-means implementation: 

- http://codereview.stackexchange.com/questions/61598/k-mean-with-numpy

In [285]:
import numpy as np
import scipy as sp

In [286]:
from sklearn.metrics.pairwise import euclidean_distances

In [287]:
def cluster_centroids(X, clusters, k):
    result = []
    
    cluster_no = 0

    for i in xrange(k):
        if (clusters == i).sum() == 0:
            continue

        centroid = X[clusters == i].mean(axis=0)
        result.append(np.array(centroid).flatten())
        cluster_no = cluster_no + 1

    return np.array(result), cluster_no

In [288]:
def kmeans(X, k, centroids=None, steps=20):
    if not centroids:
        centroids = X[np.random.choice(np.arange(X.shape[0]), size=k)]

    for _ in xrange(steps):
        D = euclidean_distances(centroids, X)
        clusters = D.argmin(axis=0)

        new_centroids, k = cluster_centroids(X, clusters, k)
        # need other stop criterion

        centroids = new_centroids

    return clusters, k

In [289]:
res, k_final = kmeans(X_red, k=300)
print k_final

295


In [290]:
evaluate.overall_purity(res)

0.12249381557435336

In [291]:
evaluate.high_purity_clusters(res, threshold=0.8)

[{'category': u'Bayer objects', 'cluster': 99, 'purity': 0.875, 'size': 56}]

In [292]:
metrics.adjusted_mutual_info_score(title_cats_code, res)

0.057591819742542349

###  K-means cosine

In [311]:
import kmeans

In [319]:
np.random.seed(300)
res, k_final = kmeans.kmeans(X_red, k=500)
print k_final

step 0... J=5.2162
step 10... J=0.0004
converged after step=19, final J=0.0015
496


In [320]:
evaluate.overall_purity(res)

0.10520550346157222

In [321]:
evaluate.high_purity_clusters(res, threshold=0.6)

[{'category': u'Bayer objects',
  'cluster': 126,
  'purity': 0.9056603773584906,
  'size': 53},
 {'category': u'Stochastic processes',
  'cluster': 157,
  'purity': 0.8,
  'size': 10}]

In [322]:
metrics.adjusted_mutual_info_score(title_cats_code, res)

0.05750327134184522

In [323]:
evaluate.print_cluster(res, 157)

size: 10

Girsanov theorem (cats: Stochastic processes, Probability theorems) F_t E E_P Ω X_t X_s W_t λ μ W_s σ Z_s Z_t F Y_t Y_s Q P T W Y X d l s r t X_0
Stochastic differential equation (cats: Differential equations, Stochastic processes, Stochastic calculus, Stochastic differential equations) C f_i g_i Z X_t X_u μ η_m σ B E D R T x_i d g k j m l n s u t X_0 y x B_t
Itō calculus (cats: Definitions of mathematical integration, Stochastic calculus) ξ_j ξ_k x_l x_m M C Y_0 F_t X_T h_k M_t t_i H_s p Ω σ_s X_t δ H_n μ X_s σ B E y F H J Y_t Y_s K x_k P α_s x_j X M_0 l c t_1 d f j f_i n π_n μ_s s t X_0 t_2 B_s B_t
Quadratic variation (cats: Stochastic processes) Y_0 Δ V_t t_k C_p Ω σ_s X_v X_t X_u X_s μ_s Y_s σ E F H Y_t M c_p X d P k m l n p s u t X_0 v B_s
Bessel process (cats: Stochastic processes) Z_t d n X_t t W_t
Novikov's condition (cats: Stochastic processes, Martingale theory) E d F Ω s l t F_t P X_t X_s X W_t W_s T
Semimartingale (cats: Stochastic processes, Martingale theory) Δ 